In [2]:
import sys
sys.path.insert(1, '/Users/linusrandud/Documents/UoM/ERP/MscDissertation/Deep-Opt')

In [3]:
import torch

from COProblems.MKP import MKP
from COProblems.QUBO import QUBO
from Models.DOAE import DOAE
from OptimAE import OptimAEHandler

In [4]:
# Highly recommended to keep as cpu for problems of size <= 100
device="cpu"
print(device)
device = torch.device(device)

cpu


In [16]:
a = [13, 23, 13, 13, 13, 13, 13, 13, 23, 13, 10, 10, 10, 20, 25, 20, 10, 15, 10, 20]
len(a)

20

In [19]:
sum(a[:10])

150

In [20]:
sum(a[10:])

150

In [21]:
problem.max_fitness

inf

## create new file for custom problem

In [216]:
import numpy as np

# Job durations
durations = [13, 23, 13, 13, 13, 13, 13, 13, 23, 13, 10, 10, 10, 20, 25, 20, 10, 15, 10, 20]

# Number of jobs
n = len(durations)

# Initialize QUBO matrix
Q = np.zeros((n, n))

# Fill the QUBO matrix
for i in range(n):
    for j in range(i, n):
        if i == j:
            Q[i, j] = durations[i] ** 2
        else:
            Q[i, j] = 2 * durations[i] * durations[j]

# Convert Q matrix to PyTorch tensor for consistency with previous code
Q_tensor = torch.tensor(Q)  # Negate for minimization

# Display the QUBO matrix
print(Q_tensor)


tensor([[ 169.,  598.,  338.,  338.,  338.,  338.,  338.,  338.,  598.,  338.,
          260.,  260.,  260.,  520.,  650.,  520.,  260.,  390.,  260.,  520.],
        [   0.,  529.,  598.,  598.,  598.,  598.,  598.,  598., 1058.,  598.,
          460.,  460.,  460.,  920., 1150.,  920.,  460.,  690.,  460.,  920.],
        [   0.,    0.,  169.,  338.,  338.,  338.,  338.,  338.,  598.,  338.,
          260.,  260.,  260.,  520.,  650.,  520.,  260.,  390.,  260.,  520.],
        [   0.,    0.,    0.,  169.,  338.,  338.,  338.,  338.,  598.,  338.,
          260.,  260.,  260.,  520.,  650.,  520.,  260.,  390.,  260.,  520.],
        [   0.,    0.,    0.,    0.,  169.,  338.,  338.,  338.,  598.,  338.,
          260.,  260.,  260.,  520.,  650.,  520.,  260.,  390.,  260.,  520.],
        [   0.,    0.,    0.,    0.,    0.,  169.,  338.,  338.,  598.,  338.,
          260.,  260.,  260.,  520.,  650.,  520.,  260.,  390.,  260.,  520.],
        [   0.,    0.,    0.,    0.,    0.,   

In [217]:
Q_tensor

tensor([[ 169.,  598.,  338.,  338.,  338.,  338.,  338.,  338.,  598.,  338.,
          260.,  260.,  260.,  520.,  650.,  520.,  260.,  390.,  260.,  520.],
        [   0.,  529.,  598.,  598.,  598.,  598.,  598.,  598., 1058.,  598.,
          460.,  460.,  460.,  920., 1150.,  920.,  460.,  690.,  460.,  920.],
        [   0.,    0.,  169.,  338.,  338.,  338.,  338.,  338.,  598.,  338.,
          260.,  260.,  260.,  520.,  650.,  520.,  260.,  390.,  260.,  520.],
        [   0.,    0.,    0.,  169.,  338.,  338.,  338.,  338.,  598.,  338.,
          260.,  260.,  260.,  520.,  650.,  520.,  260.,  390.,  260.,  520.],
        [   0.,    0.,    0.,    0.,  169.,  338.,  338.,  338.,  598.,  338.,
          260.,  260.,  260.,  520.,  650.,  520.,  260.,  390.,  260.,  520.],
        [   0.,    0.,    0.,    0.,    0.,  169.,  338.,  338.,  598.,  338.,
          260.,  260.,  260.,  520.,  650.,  520.,  260.,  390.,  260.,  520.],
        [   0.,    0.,    0.,    0.,    0.,   

## save problem

In [218]:
# Generate the text file content
lines = []
lines.append("1\n")  # Number of problem instances

# Collect non-zero entries
non_zero_entries = []
for i in range(n):
    for j in range(i, n):
        if Q[i][j] != 0:
            non_zero_entries.append((i + 1, j + 1, int(Q[i][j])))

# Add the number of variables and number of non-zero entries
lines.append(f"{n} {len(non_zero_entries)}\n")

# Add the non-zero entries to lines
for entry in non_zero_entries:
    i, j, value = entry
    lines.append(f"{i} {j} {value}\n")

# Write to file
with open("../data/qubo/qubo_problem.txt", "w") as f:
    f.writelines(lines)

## load problem

In [5]:
problem = QUBO("../data/qubo/qubo_problem.txt", 0, device)

Instance has been loaded


In [6]:
problem

In [7]:
dir(problem)

['Q',
 '__abstractmethods__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 'device',
 'fitness',
 'is_valid',
 'max_fitness',
 'random_solution',
 'repair']

In [8]:
problem.Q

tensor([[ 169.,  598.,  338.,  338.,  338.,  338.,  338.,  338.,  598.,  338.,
          260.,  260.,  260.,  520.,  650.,  520.,  260.,  390.,  260.,  520.],
        [ 598.,  529.,  598.,  598.,  598.,  598.,  598.,  598., 1058.,  598.,
          460.,  460.,  460.,  920., 1150.,  920.,  460.,  690.,  460.,  920.],
        [ 338.,  598.,  169.,  338.,  338.,  338.,  338.,  338.,  598.,  338.,
          260.,  260.,  260.,  520.,  650.,  520.,  260.,  390.,  260.,  520.],
        [ 338.,  598.,  338.,  169.,  338.,  338.,  338.,  338.,  598.,  338.,
          260.,  260.,  260.,  520.,  650.,  520.,  260.,  390.,  260.,  520.],
        [ 338.,  598.,  338.,  338.,  169.,  338.,  338.,  338.,  598.,  338.,
          260.,  260.,  260.,  520.,  650.,  520.,  260.,  390.,  260.,  520.],
        [ 338.,  598.,  338.,  338.,  338.,  169.,  338.,  338.,  598.,  338.,
          260.,  260.,  260.,  520.,  650.,  520.,  260.,  390.,  260.,  520.],
        [ 338.,  598.,  338.,  338.,  338.,  3

In [9]:
torch.count_nonzero(problem.Q, dim=0), torch.count_nonzero(problem.Q, dim=1)

(tensor([20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20,
         20, 20]),
 tensor([20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20,
         20, 20]))

In [10]:
torch.count_nonzero(Q_tensor, dim=0), torch.count_nonzero(Q_tensor, dim=1)

NameError: name 'Q_tensor' is not defined

In [11]:
Q_tensor[1]

NameError: name 'Q_tensor' is not defined

In [12]:
problem.Q[1]

tensor([ 598.,  529.,  598.,  598.,  598.,  598.,  598.,  598., 1058.,  598.,
         460.,  460.,  460.,  920., 1150.,  920.,  460.,  690.,  460.,  920.])

In [13]:
change_tolerance = 20
problem_size = 20
pop_size = 100

dropout_prob = 0.2
# 1000 bit QUBO l1 and l2:
# l1_coef = 0.0000025
# l2_coef = 0.0000025
# 500 bit QUBO l1 and l2:
# l1_coef = 0.000005
# l2_coef = 0.000005
# 100 bit l1 and l2
l1_coef = 0.0001
l2_coef = 0.0001
lr = 0.002
compression_ratio = 0.8
model = DOAE(problem_size, dropout_prob, device)
hidden_size = problem_size
handler = OptimAEHandler(model, problem, device)

population, fitnesses = handler.generate_population(pop_size)
population, fitnesses, _, _ = handler.hillclimb(population, fitnesses, change_tolerance)
handler.print_statistics(fitnesses)

total_eval = 0
max_depth = 6
depth = 0

while True:
    if depth < max_depth:
        hidden_size = round(hidden_size * compression_ratio)
        model.transition(hidden_size)
        depth += 1
        optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=l2_coef)
    print("Learning from population")
    # Learing with the entire population as a batch is technically not the best from a machine learning perspective,
    # but does not seem to have a massive impact on solution quality whilst also increasing learning speed significantly.
    handler.learn_from_population(population, optimizer, l1_coef=l1_coef, batch_size=pop_size)
    print("Optimising population")
    population, fitnesses, evaluations, done = handler.optimise_solutions(
        population, fitnesses, change_tolerance, encode=True, repair_solutions=True, deepest_only=False
    )
    handler.print_statistics(fitnesses)
    total_eval += evaluations
    print("Evaluations: {}".format(total_eval))
    
    # Uncomment lines below to print out best solution at every transition
    # best_i = torch.argmax(fitnesses)
    # print("Best solution - fitness = {}".format(fitnesses[best_i].item()))
    # print(population[best_i].tolist())
    if done:
        break

Max pop fitness: 175040.0, Mean pop fitness : 163818.625
Learning from population
Optimising population
Max pop fitness: 175040.0, Mean pop fitness : 165063.828125
Evaluations: 2131
Learning from population
Optimising population
Max pop fitness: 175040.0, Mean pop fitness : 165063.828125
Evaluations: 6331
Learning from population
Optimising population
Max pop fitness: 175040.0, Mean pop fitness : 165063.828125
Evaluations: 12631
Learning from population
Optimising population
Max pop fitness: 175040.0, Mean pop fitness : 165063.828125
Evaluations: 21031
Learning from population
Optimising population
Max pop fitness: 175040.0, Mean pop fitness : 165063.828125
Evaluations: 31531
Learning from population
Optimising population
Max pop fitness: 175040.0, Mean pop fitness : 165063.828125
Evaluations: 44131
Learning from population
Optimising population
Max pop fitness: 175040.0, Mean pop fitness : 165063.828125
Evaluations: 56731
Learning from population
Optimising population
Max pop fitness:

KeyboardInterrupt: 

In [15]:
population[0]

tensor([ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1., -1.,
         1.,  1.,  1.,  1.,  1.,  1.])